# Parameterized Circuits - Complete Guide for Qiskit Certification

**Section 3: Create Circuits - Part 3**

> **Exam Weight**: Part of 18% (HIGHEST WEIGHT!) | **Must Master**: ✅✅✅

This notebook covers parameterized circuits - the foundation of variational quantum algorithms like VQE and QAOA!

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Create symbolic parameters with `Parameter` and `ParameterVector`
- Bind parameters using `assign_parameters()`
- Understand parameter lifecycle (create → use → bind)
- Build VQE ansatz patterns
- Identify exam traps with parameter binding

---

## 💡 Conceptual Deep Dive: The Recipe Analogy

**Parameterized Circuit = Recipe Template**

| Recipe Template | Parameterized Circuit |
|-----------------|----------------------|
| **"Add X cups of flour"** | `qc.ry(theta, 0)` |
| **Specific amount (2 cups)** | `assign_parameters({theta: 1.57})` |
| **Variables in recipe** | Parameters |
| **Final cooked dish** | Bound circuit (executable) |

```
"A parameterized circuit is like a recipe with blanks:
 'Rotate by ___ radians' 
 You fill in the values before execution!"
                                    - VQE Cooking Guide
```

**Key Insight**:
- Parameters are symbolic (variables, not numbers)
- Circuit is a template until parameters are bound
- Same template can be reused with different values
- Essential for optimization loops (VQE, QAOA)

---

## 🔬 Critical Exam Concept: Parameter Binding

**This is the #1 tested concept for VQE questions!**

```python
# CREATE: Define symbolic parameter
theta = Parameter('θ')

# USE: Put in circuit (still symbolic)
qc.ry(theta, 0)  # Circuit NOT executable yet!

# BIND: Replace symbol with value
bound_qc = qc.assign_parameters({theta: np.pi/4})
# NOW circuit is executable!
```

**⚠️ EXAM TRAP**: `assign_parameters()` returns a NEW circuit!
```python
# ❌ WRONG
qc.assign_parameters({theta: 0.5})  # Original unchanged!

# ✅ CORRECT  
bound_qc = qc.assign_parameters({theta: 0.5})  # Capture result!
```

---

## Setup

Import required libraries:

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
import numpy as np

print("✅ Imports successful!")

## Setup

Import required libraries:

## Part 1: Single Parameter (EXAM CRITICAL!)

In [ ]:
# Create parameter
theta = Parameter('θ')
print(f"Parameter: {theta}")
print(f"Name: {theta.name}")

# Use in circuit
qc = QuantumCircuit(1)
qc.ry(theta, 0)

print(f"\nCircuit with parameter:")
print(qc.draw())
print(f"\nParameters in circuit: {qc.parameters}")
print(f"Number of parameters: {qc.num_parameters}")

print("\n✅ Circuit is symbolic (not executable yet!)")

### assign_parameters() - MOST TESTED METHOD!

In [ ]:
# Bind parameter to value
bound_circuit = qc.assign_parameters({theta: np.pi/4})

print("Bound circuit (θ = π/4):")
print(bound_circuit.draw())
print(f"\nParameters: {bound_circuit.parameters}")
print("✅ No parameters left - circuit is executable!")

In [ ]:
# Bind to multiple values (VQE pattern!)
angles = [0, np.pi/4, np.pi/2, np.pi]

print("Binding to multiple values:")
for angle in angles:
    bound = qc.assign_parameters({theta: angle})
    print(f"\nθ = {angle:.4f}:")
    print(bound.draw())

print("\n🎯 EXAM PATTERN: VQE loop binds different values each iteration!")

### ⚠️ EXAM TRAP: assign_parameters() Returns NEW Circuit!

```python
# ❌ WRONG - Original circuit unchanged
qc.assign_parameters({theta: 0.5})
# qc still has parameter!

# ✅ CORRECT - Capture returned circuit
bound_qc = qc.assign_parameters({theta: 0.5})
# bound_qc has no parameters
```

## Part 2: ParameterVector - Multiple Parameters

In [ ]:
# Create parameter vector
params = ParameterVector('θ', 3)

print(f"ParameterVector: {params}")
print(f"Individual parameters: {params[0]}, {params[1]}, {params[2]}")

# Use in circuit
qc = QuantumCircuit(3)
for i in range(3):
    qc.ry(params[i], i)

print(f"\nCircuit with parameter vector:")
print(qc.draw())
print(f"\nParameters: {qc.parameters}")


In [ ]:
# Bind vector with list
values = [np.pi/2, np.pi/3, np.pi/4]
bounded_qc = qc.assign_parameters({params: values})

print("Bound with list [π/4, π/3, π/2]:")
print(bound_qc.draw())
print("\n✅ ParameterVector more efficient than multiple Parameter objects!")

## Part 3: VQE Ansatz Pattern (GUARANTEED EXAM QUESTION!)

**This exact pattern appears on every exam!**

In [ ]:
def create_vqe_ansatz(n_qubits, depth):
    """
    Hardware-efficient ansatz for VQE
    
    EXAM PATTERN - MEMORIZE THIS STRUCTURE:
    1. Create ParameterVector
    2. Add rotation layers
    3. Add entangling layers
    4. Repeat for depth
    """
    qc = QuantumCircuit(n_qubits)
    
    # Parameters: 2 angles per qubit per layer
    params = ParameterVector('θ', n_qubits * depth * 2)
    idx = 0
    
    for d in range(depth):
        # Rotation layer (Y rotations)
        for i in range(n_qubits):
            qc.ry(params[idx], i)
            idx += 1
        
        # Rotation layer (Z rotations)
        for i in range(n_qubits):
            qc.rz(params[idx], i)
            idx += 1
        
        # Entangling layer
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)
    
    return qc, params

# Create 3-qubit, depth-2 ansatz
ansatz, params = create_vqe_ansatz(3, 2)

print("VQE Ansatz (3 qubits, depth 2):")
print(ansatz.draw())
print(f"\nTotal parameters: {len(params)}")
print(f"Expected: 3 qubits × 2 depths × 2 angles = {3*2*2}")
print("\n🎯 This pattern appears on EVERY exam!")

In [ ]:
# VQE optimization loop pattern
print("VQE Optimization Loop (EXAM PATTERN):")
print("="*50)

# Initial parameters
current_params = np.random.random(len(params)) * 2 * np.pi

print("\nIteration pattern:")
print("""
for iteration in range(max_iterations):
    # 1. Bind parameters
    bound_circuit = ansatz.assign_parameters({params: current_params})
    
    # 2. Run on quantum computer (Estimator)
    energy = estimator.run([bound_circuit], [hamiltonian]).result()
    
    # 3. Optimize parameters
    current_params = optimizer.step(energy)
    
    # 4. Check convergence
    if converged:
        break
""")

# Demonstrate binding
bound_ansatz = ansatz.assign_parameters({params: current_params})
print("\nBound ansatz (executable):")
print(bound_ansatz.draw())
print(f"Parameters remaining: {len(bound_ansatz.parameters)}")

## Part 4: Parameter Expressions (Advanced)

In [ ]:
# Parameter expressions
theta = Parameter('θ')
phi = Parameter('φ')

qc = QuantumCircuit(1)
qc.rx(theta, 0)
qc.ry(2*theta, 0)        # Expression: 2θ
qc.rz(theta + phi, 0)    # Expression: θ + φ

print("Circuit with parameter expressions:")
print(qc.draw())

# Bind
bound = qc.assign_parameters({theta: np.pi/4, phi: np.pi/6})
print("\nBound circuit:")
print(bound.draw())

## Part 5: Partial Binding

In [ ]:
# Partial binding - bind some parameters
theta = Parameter('θ')
phi = Parameter('φ')

qc = QuantumCircuit(1)
qc.rx(theta, 0)
qc.ry(phi, 0)

print("Original circuit:")
print(qc.draw())
print(f"Parameters: {qc.parameters}")

# Bind only theta
partial = qc.assign_parameters({theta: np.pi/4})

print("\nAfter binding θ only:")
print(partial.draw())
print(f"Parameters remaining: {partial.parameters}")
print("\n✅ Can bind parameters incrementally!")

## 📝 Practice Questions

### Question 1: assign_parameters()

**What does assign_parameters() return?**

A) None (modifies circuit in-place)  
B) A new circuit with bound parameters  
C) A list of parameter values  
D) The original circuit

<details>
<summary>Answer</summary>

**B) A new circuit with bound parameters**

```python
# Returns NEW circuit
bound_qc = qc.assign_parameters({theta: 0.5})

# Original unchanged
print(qc.parameters)  # Still has theta!
print(bound_qc.parameters)  # Empty set()
```

**CRITICAL**: Must capture the return value!
</details>

---

### Question 2: ParameterVector

**How many parameters in `ParameterVector('θ', 5)`?**

A) 1  
B) 4  
C) 5  
D) 6

<details>
<summary>Answer</summary>

**C) 5**

```python
params = ParameterVector('θ', 5)
# Creates: θ[0], θ[1], θ[2], θ[3], θ[4]
# Total: 5 parameters
```

The second argument is the LENGTH of the vector.
</details>

---

### Question 3: VQE Pattern

**In VQE, when do you bind parameters?**

A) Once before starting optimization  
B) Every iteration with current parameter values  
C) After optimization completes  
D) Never - use symbolic parameters

<details>
<summary>Answer</summary>

**B) Every iteration with current parameter values**

VQE loop:
```python
for iteration in range(max_iter):
    # Bind current parameters
    bound = ansatz.assign_parameters({params: current_values})
    
    # Measure energy
    energy = estimator.run([bound], [hamiltonian]).result()
    
    # Update parameters
    current_values = optimizer.step(energy)
```

Must bind every iteration because parameters change!
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **Parameter** - Symbolic placeholder
   - `theta = Parameter('θ')`
   - Use in rotation gates: `qc.ry(theta, 0)`

2. **ParameterVector** - Multiple parameters efficiently
   - `params = ParameterVector('θ', n)`
   - Access: `params[0]`, `params[1]`, ...

3. **assign_parameters()** - Bind to values (MOST TESTED!)
   - Returns NEW circuit
   - Dict: `{theta: 0.5}` or `{params: [0.1, 0.2, 0.3]}`
   - Original circuit unchanged

4. **VQE Pattern** - GUARANTEED exam question
   - Create parameterized ansatz
   - Loop: bind → measure → optimize
   - Use with Estimator primitive

### Critical Exam Facts

- ✅ assign_parameters() **returns NEW circuit**
- ✅ ParameterVector more efficient than multiple Parameters
- ✅ Circuits with parameters are **not executable**
- ✅ VQE binds parameters **every iteration**
- ✅ Can use expressions: `2*theta`, `theta + phi`
- ✅ Partial binding possible (bind some, keep others)

### Exam Patterns

**Q**: How to create parameterized circuit?  
**A**: `theta = Parameter('θ')`, then `qc.ry(theta, 0)`

**Q**: How to bind parameter?  
**A**: `bound = qc.assign_parameters({theta: value})`

**Q**: Does assign_parameters() modify original?  
**A**: No! Returns new circuit

**Q**: VQE workflow?  
**A**: Create ansatz → loop(bind → measure → optimize)

### Mnemonic

🧠 **"Parameters are Placeholders - Assign to Execute!"**

**Next**: Circuit Library (QFT, ansätze) and Classical Control (c_if)!